In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [3]:
# 데이터 로드
data = pd.read_csv("C:\\Users\\LG\\Downloads\\BP_data.csv")

In [6]:
data.isnull().sum()
data = data.fillna(0)

In [8]:
# 특성과 타겟 분리
X = data.drop(columns=['Blood_Pressure_Abnormality']).values
y = data['Blood_Pressure_Abnormality'].values.astype(np.int64)  # 분류이므로 int64

In [10]:
# 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [12]:
# train/test 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Tensor 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [16]:
# 이진 분류 모델 정의
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(14, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)  # 이진 분류: 클래스 2개
        )

    def forward(self, x):
        return self.model(x)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ClassificationModel().to(device)
criterion = nn.CrossEntropyLoss()  # 분류용 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
# 학습 루프
model.train()
for epoch in range(50):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 0.6422
Epoch 2, Loss: 0.5420
Epoch 3, Loss: 0.5027
Epoch 4, Loss: 0.4769
Epoch 5, Loss: 0.4525
Epoch 6, Loss: 0.4273
Epoch 7, Loss: 0.4039
Epoch 8, Loss: 0.3846
Epoch 9, Loss: 0.3627
Epoch 10, Loss: 0.3480
Epoch 11, Loss: 0.3377
Epoch 12, Loss: 0.3290
Epoch 13, Loss: 0.3220
Epoch 14, Loss: 0.3134
Epoch 15, Loss: 0.3078
Epoch 16, Loss: 0.3059
Epoch 17, Loss: 0.2967
Epoch 18, Loss: 0.2940
Epoch 19, Loss: 0.2874
Epoch 20, Loss: 0.2833
Epoch 21, Loss: 0.2817
Epoch 22, Loss: 0.2781
Epoch 23, Loss: 0.2689
Epoch 24, Loss: 0.2679
Epoch 25, Loss: 0.2650
Epoch 26, Loss: 0.2594
Epoch 27, Loss: 0.2609
Epoch 28, Loss: 0.2570
Epoch 29, Loss: 0.2507
Epoch 30, Loss: 0.2447
Epoch 31, Loss: 0.2405
Epoch 32, Loss: 0.2416
Epoch 33, Loss: 0.2344
Epoch 34, Loss: 0.2325
Epoch 35, Loss: 0.2282
Epoch 36, Loss: 0.2257
Epoch 37, Loss: 0.2228
Epoch 38, Loss: 0.2193
Epoch 39, Loss: 0.2106
Epoch 40, Loss: 0.2102
Epoch 41, Loss: 0.2077
Epoch 42, Loss: 0.2058
Epoch 43, Loss: 0.1984
Epoch 44, Loss: 0.19

In [22]:
# 평가
model.eval()
all_preds = []
with torch.no_grad():
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)

In [24]:
accuracy = accuracy_score(y_test, all_preds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 84.00%
